###LLMを使った、 「吾輩は猫である」に基づいたRetrieval Augmented Generation(RAG)の実装


In [ ]:
!pip install vllm
!pip install sentence_transformers
!pip install gradio

In [2]:
import torch
from vllm import LLM, SamplingParams
from sentence_transformers import SentenceTransformer

def load_model(model_name):
    model = LLM(model=model_name, quantization="awq", gpu_memory_utilization=0.6)
    tokenizer = model.get_tokenizer()
    return model, tokenizer

def load_embedding_model(embedding_model_name):
    embedding_model = SentenceTransformer(embedding_model_name)
    return embedding_model

In [3]:
import pandas as pd

def load_text(model):
    df = pd.read_csv('RAG/neco.txt', header=None, names=['content'])
    passages = ['passage: ' + content for content in df.content.tolist()]
    passage_embeddings = model.encode(passages, normalize_embeddings=True)
    return df, passages, passage_embeddings

def retrieve_text(df, passages, passage_embeddings, query, model, verbose=False):
    query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
    scores = (query_embeddings @ passage_embeddings.T) * 100

    top_k = 3
    top_k_idx = scores[0].argsort()[::-1][:top_k]

    retrieved_text = f"""{df.content.tolist()[top_k_idx[0]][:20]}
    {df.content.tolist()[top_k_idx[1]][:20]}
    {df.content.tolist()[top_k_idx[2]][:20]}
    """

    if verbose:
        # 検索結果上位3件: cores[0][scores[0].argsort()[::-1][:3]]
        return retrieved_text, scores

    return retrieved_text, ''

In [4]:
def format_text(query, retrieved_text, is_retrieval):
    DEFAULT_SYSTEM_PROMPT = "あなたは誠実で優秀な日本人のアシスタントです。特に指示が無い場合は、常に日本語で回答してください。"

    if is_retrieval:
        text = f"""{retrieved_text}
        上記の文章のみをもとにして質問に回答してください。一歩ずつ考えましょう。
        質問: {query}
        回答:"""
    else:
        text=f"""質問: {query}
        回答:"""

    messages = [
        {"role": "system", "content": DEFAULT_SYSTEM_PROMPT},
        {"role": "user", "content": text},
    ]

    return messages

In [5]:
def generate_text(model, tokenizer, messages):
    sampling_params = SamplingParams(temperature=0.6, top_p=0.9, max_tokens=1000)
    prompt = [tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )]

    output = model.generate(prompt, sampling_params)

    return output[0].outputs[0].text

In [ ]:
model_name = 'elyza/Llama-3-ELYZA-JP-8B-AWQ'  # https://huggingface.co/elyza/Llama-3-ELYZA-JP-8B
embedding_model_name = 'intfloat/multilingual-e5-large'  # https://huggingface.co/intfloat/multilingual-e5-large

model, tokenizer = load_model(model_name)
embedding_model = load_embedding_model(embedding_model_name)
df, passages, passage_embeddings = load_text(embedding_model)

is_retrieval = True
verbose = True

def rag(query, is_retrieval):
    if is_retrieval:
        retrieved_text, scores = retrieve_text(df, passages, passage_embeddings, query, embedding_model, verbose)
    else:
        retrieved_text = None

    messages = format_text(query, retrieved_text, is_retrieval)
    output = generate_text(model, tokenizer, messages)
    return output, scores

In [ ]:
output, scores = rag("吾輩が指すものは何ですか。", is_retrieval)

In [10]:
# 検索結果上位3件
print('score: ', scores[0][scores[0].argsort()[::-1][0]])
print(passages[scores[0].argsort()[::-1][0]])
print('score: ', scores[0][scores[0].argsort()[::-1][1]])
print(passages[scores[0].argsort()[::-1][1]])
print('score: ', scores[0][scores[0].argsort()[::-1][2]])
print(passages[scores[0].argsort()[::-1][2]])

score:  86.883286
passage: 吾輩（わがはい）は猫である。名前はまだ無い。
score:  85.5455
passage: 吾輩はまた少々休養を要する。主人と多々良君が上野公園でどんな真似をして、芋坂で団子を幾皿食ったかその辺の逸事は探偵の必要もなし、また尾行（びこう）する勇気もないからずっと略してその間（あいだ）休養せんければならん。休養は万物の旻天（びんてん）から要求してしかるべき権利である。この世に生息すべき義務を有して蠢動（しゅんどう）する者は、生息の義務を果すために休養を得ねばならぬ。もし神ありて汝（なんじ）は働くために生れたり寝るために生れたるに非ずと云わば吾輩はこれに答えて云わん、吾輩は仰せのごとく働くために生れたり故に働くために休養を乞うと。主人のごとく器械に不平を吹き込んだまでの木強漢（ぼくきょうかん）ですら、時々は日曜以外に自弁休養をやるではないか。多感多恨にして日夜心神を労する吾輩ごとき者は仮令（たとい）猫といえども主人以上に休養を要するは勿論の事である。ただ先刻（さっき）多々良君が吾輩を目して休養以外に何等の能もない贅物（ぜいぶつ）のごとくに罵（ののし）ったのは少々気掛りである。とかく物象（ぶっしょう）にのみ使役せらるる俗人は、五感の刺激以外に何等の活動もないので、他を評価するのでも形骸以外に渉（わた）らんのは厄介である。何でも尻でも端折（はしょ）って、汗でも出さないと働らいていないように考えている。達磨（だるま）と云う坊さんは足の腐るまで座禅をして澄ましていたと云うが、仮令（たとい）壁の隙（すき）から蔦（つた）が這い込んで大師の眼口を塞（ふさ）ぐまで動かないにしろ、寝ているんでも死んでいるんでもない。頭の中は常に活動して、廓然無聖（かくねんむしょう）などと乙な理窟を考え込んでいる。儒家にも静坐の工夫と云うのがあるそうだ。これだって一室の中（うち）に閉居して安閑と躄（いざり）の修行をするのではない。脳中の活力は人一倍熾（さかん）に燃えている。ただ外見上は至極沈静端粛の態（てい）であるから、天下の凡眼はこれらの知識巨匠をもって昏睡仮死（こんすいかし）の庸人（ようじん）と見做（みな）して無用の長物とか穀潰（ごくつぶ）しとか入らざる誹謗（ひぼう）の声を立てるのである。これらの凡眼は皆形を見て心を見ざる不具なる視覚を有して生

In [11]:
# 回答
print(output)

この短い文章から考えるに、吾輩が指すものは「猫」です。


In [14]:
import gradio as gr

def handle_submit(query, chat_history):
    bot_message, score = rag(query, True)
    chat_history.append((query, bot_message))
    return "", chat_history

js = """
function createGradioAnimation() {
    var container = document.createElement('div');
    container.id = 'gradio-animation';
    container.style.fontSize = '2em';
    container.style.fontWeight = 'bold';
    container.style.textAlign = 'center';
    container.style.marginBottom = '20px';

    var text = 'Welcome';
    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.5s';
                letter.innerText = text[i];

                container.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 50);
            }, i * 250);
        })(i);
    }

    var gradioContainer = document.querySelector('.gradio-container');
    gradioContainer.insertBefore(container, gradioContainer.firstChild);

    return 'Animation created';
}
"""

with gr.Blocks(theme="Ajaxon6255/Emerald_Isle", js=js) as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(handle_submit, [msg, chatbot], [msg, chatbot])


demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0c184e2a7bdeae0a83.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


##参考文献
In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083